# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/test-bed-fltk-jerrit/fltk`
    * Push to your gcr.io repository `docker push gcr.io/test-bed-fltk-jerrit/fltk`


With that setup, first set some variables used throughout the experiment.


In [81]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="fltk-flint" # changed
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
EXPERIMENT_POOL1="fltk-pool-1" # changed
EXPERIMENT_POOL2="fltk-pool-2" # changed
EXPERIMENT_POOL=$EXPERIMENT_POOL2
REGION="us-central1-c"

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Scale the default-node-pool up.

In [79]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 1 --region $REGION --quiet
     

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-jerrit/zones/us-central1-c/clusters/fltk-testbed-cluster].


In [82]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
     --num-nodes 4 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-jerrit/zones/us-central1-c/clusters/fltk-testbed-cluster].


## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

In [4]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
# kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

helm uninstall -n test flearner

W1011 09:21:11.908953    4448 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: flearner: release: not found


: 1

## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [110]:
helm upgrade --install -n test extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

W1011 11:33:18.351805   14409 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Release "extractor" does not exist. Installing it now.
NAME: extractor
LAST DEPLOYED: Tue Oct 11 11:33:20 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [84]:
EXPERIMENT_FILE="../configs/distributed_tasks/all_models_experiment.json"
CLUSTER_CONFIG="../configs/vision_transformer_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [85]:
helm uninstall -n test experiment-orchestrator

W1011 11:15:16.236316   13319 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: experiment-orchestrator: release: not found


: 1

In [111]:
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

W1011 11:33:31.122381   14427 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME: experiment-orchestrator
LAST DEPLOYED: Tue Oct 11 11:33:32 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [135]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

10-11-2022 09:33:40 root         INFO     Loading file config/configuration.fltk.json
No argument path is provided.
10-11-2022 09:33:40 root         INFO     Starting in cluster mode.
10-11-2022 09:33:40 root         INFO     Starting with experiment replication: 0 with seed: 42
10-11-2022 09:33:40 root         INFO     Starting as Orchestrator
10-11-2022 09:33:40 root         INFO     Starting Orchestrator, initializing resources....
10-11-2022 09:33:40 root         INFO     Loading in cluster configuration file
10-11-2022 09:33:40 root         INFO     Pointing configuration to in cluster configuration.
10-11-2022 09:33:40 root         INFO     Starting cluster manager
10-11-2022 09:33:40 ClusterManager INFO     Spinning up cluster manager...
10-11-2022 09:33:40 ResourceWatchDog INFO     Starting resource watchdog
10-11-2022 09:33:40 ResourceWatchDog INFO     Fetching node information of cluster...
10-11-2022 09:33:40 root         INFO     Starting arrival generator
10-11-2022 09:33:

10-11-2022 09:36:32 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:36:36 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 09:36:37 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:36:42 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:36:47 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:36:47 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 09:36:52 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:36:53 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 09:36:57 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 

10-11-2022 09:40:33 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:40:38 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:40:39 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 09:40:43 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:40:48 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:40:50 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 09:40:53 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:40:58 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:41:01 ResourceWatchDog INFO     Fetching pod information of 

10-11-2022 09:44:34 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:44:39 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:44:43 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 09:44:44 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:44:49 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:44:54 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 09:44:54 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:44:59 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:45:04 root         INFO     Waiting for 9db41c7d-0c78-4fed-a

10-11-2022 09:48:41 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:48:46 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:48:46 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 09:48:51 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:48:51 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 09:48:56 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:48:57 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 09:49:01 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 09:49:06 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 

...

10-11-2022 10:17:00 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:17:05 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:17:05 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 10:17:10 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:17:11 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 10:17:15 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:17:20 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:17:22 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 10:17:25 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 

10-11-2022 10:21:03 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 10:21:06 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:21:11 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:21:13 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 10:21:16 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:21:21 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:21:25 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 10:21:26 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:21:30 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 10:21:31 root      

10-11-2022 10:25:06 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 10:25:07 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:25:12 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:25:17 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 10:25:17 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:25:22 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:25:27 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:25:28 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 10:25:32 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 

10-11-2022 10:29:10 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 10:29:14 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:29:16 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 10:29:19 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:29:22 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 10:29:24 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:29:29 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:29:33 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 10:29:34 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:29:39 root      

10-11-2022 10:33:14 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 10:33:15 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:33:20 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:33:25 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:33:25 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 10:33:30 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:33:35 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 0 arrivals
10-11-2022 10:33:36 ResourceWatchDog INFO     Fetching pod information of cluster...
10-11-2022 10:33:40 root         INFO     Waiting for 9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3 to complete, 0 pending, 

In [144]:
kubectl logs -n test trainjob-0749f555-8789-4638-851a-086ca4ecaba6-master-0 

#kubectl logs -n test trainjob-40f1c0f7-50f3-45a3-b8c2-23cfbb423f6c-master-0 

#kubectl logs -n test trainjob-9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3-master-0 

#kubectl logs -n test trainjob-b1339347-cbc3-405c-b577-9b897369afe1-master-0 

10-11-2022 09:33:51 root         INFO     Loading file config/configuration.fltk.json
10-11-2022 09:33:51 root         INFO     Starting in client mode
10-11-2022 09:33:51 root         INFO     Starting with host=trainjob-0749f555-8789-4638-851a-086ca4ecaba6-master-0 and port=23456
10-11-2022 09:33:51 root         INFO     Starting Creating client with 0
10-11-2022 09:33:51 Client-0-0749f555-8789-4638-851a-086ca4ecaba6 INFO     Initializing learning client
10-11-2022 09:33:51 root         INFO     Getting net: Nets.swin_flowers
10-11-2022 09:33:52 Client-0-0749f555-8789-4638-851a-086ca4ecaba6 INFO     Preparing learner model with distributed=False
10-11-2022 09:33:52 Client-0-0749f555-8789-4638-851a-086ca4ecaba6 INFO     Starting training of model Nets.swin_flowers with batch size 25 and max epochs 90.
10-11-2022 09:33:58 Client-0-0749f555-8789-4638-851a-086ca4ecaba6 INFO     [1,     0] loss: 0.096
/opt/federation-lab/fltk/nets/util/evaluation.py:8: RuntimeWarning: invalid value encoun

## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [21]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")

kubectl cp -n test $EXTRACTOR_POD_NAME:/opt/federation-lab/logging ./logging

tar: Removing leading `/' from member names


# Cleanup

## Removing orchestrator

In [107]:
helm uninstall -n test experiment-orchestrator

W1011 11:32:16.137238   14252 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [108]:
helm uninstall extractor -n test

W1011 11:32:20.271793   14266 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [109]:
# This will remove all information and logs as well.
kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

pytorchjob.kubeflow.org "trainjob-4fb145c6-37af-4d81-8a1e-4ac28253f4d8" deleted
pytorchjob.kubeflow.org "trainjob-639793cb-b02b-4209-b19e-97cfd1c0bf48" deleted
pytorchjob.kubeflow.org "trainjob-c5c2fbb4-b134-445a-a3b9-cb07d827e759" deleted
pytorchjob.kubeflow.org "trainjob-ecf774d6-16d6-4cd9-a593-0b9d2a0d1740" deleted


In [77]:
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-jerrit/zones/us-central1-c/clusters/fltk-testbed-cluster].
Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-jerrit/zones/us-central1-c/clusters/fltk-testbed-cluster].
